Практическое задание.
KNN и Линейная регрессия

Задание 1 (30%). Сгенерировать случайным образом 3 набора данных (см. картинки). Количество точек в каждом от 100 до 200. Цветом обозначены метки классов.

Для каждого набора данных:
* Разделить датасет на train / test(80/20)
* Обучить и протестировать KNN
* Построить графики точности классификации (на train/test выборках) при разных k(1...8)
* Пользуясь стандартными методами из scikit-learn вычислить точность, полноту и F1-меру, построить матрицу расхождений 

Задание 2 (20%). Метрики
* 25 объектов классифицированы на 3 класса
* C="Cat"
* F="Fish"    
* H="Hen"
* истинные значения
* y_true = [C,C,C,C,C,C, F,F,F,F,F,F,F,F,F,F, H,H,H,H,H,H,H,H,H]
* результат системы
* y_pred = [C,C,C,C,H,F, C,C,C,C,C,C,H,H,F,F, C,C,C,H,H,H,H,H,H]

Написать код на python, который воспроизводит вывод, представленный ниже(не используя сторонние библиотеки, кроме numpy):

In [3]:
import numpy as np

C = "Cat"
F = "Fish"
H = "Hen"
# Истинные значения
y_true = [C,C,C,C,C,C, F,F,F,F,F,F,F,F,F,F, H,H,H,H,H,H,H,H,H]
# Результат системы
y_pred = [C,C,C,C,H,F, C,C,C,C,C,C,H,H,F,F, C,C,C,H,H,H,H,H,H]

# Функция для расчета матрицы ошибок
def confusion_matrix(y_true, y_pred, classes):
    num_classes = len(classes)
    matrix = np.zeros((num_classes, num_classes), dtype=int)
    
    for i in range(len(y_true)):
        true_class = classes.index(y_true[i])
        pred_class = classes.index(y_pred[i])
        matrix[true_class][pred_class] += 1

    return matrix

# Функция для расчета precision, recall, и f1-score
def calculate_metrics(matrix):
    num_classes = len(matrix)
    precision = []
    recall = []
    f1_score = []

    for i in range(num_classes):
        tp = matrix[i][i]
        fp = np.sum(matrix[:, i]) - tp
        fn = np.sum(matrix[i, :]) - tp
        tn = np.sum(matrix) - (tp + fp + fn)

        class_precision = tp / (tp + fp) if tp + fp != 0 else 0
        class_recall = tp / (tp + fn) if tp + fn != 0 else 0
        class_f1_score = (2 * class_precision * class_recall) / (class_precision + class_recall) if (class_precision + class_recall) != 0 else 0

        precision.append(class_precision)
        recall.append(class_recall)
        f1_score.append(class_f1_score)

    return precision, recall, f1_score

# Определение классов
classes = ["Cat", "Fish", "Hen"]

# Вычисление матрицы ошибок
confusion_matrix_result = confusion_matrix(y_true, y_pred, classes)

# Вывод матрицы ошибок
print("Confusion Matrix:")
print(confusion_matrix_result)
print()

# Расчет precision, recall, и f1-score
precision, recall, f1_score = calculate_metrics(confusion_matrix_result)

# Расчет support (true positive + false negative)
support = np.sum(confusion_matrix_result, axis=1)

# Вычисление accuracy
accuracy = np.sum(np.diag(confusion_matrix_result)) / np.sum(confusion_matrix_result)

# Вычисление macro average
macro_avg_precision = np.mean(precision)
macro_avg_recall = np.mean(recall)
macro_avg_f1_score = np.mean(f1_score)

# Вычисление weighted average
weighted_avg_precision = np.sum(precision * support) / np.sum(support)
weighted_avg_recall = np.sum(recall * support) / np.sum(support)
weighted_avg_f1_score = np.sum(f1_score * support) / np.sum(support)

# Вывод результата работы программы в нужном формате
print("{:<15} {:<12} {:<12} {:<12} {:<12}".format("", "precision", "recall", "f1-score", "support"))
print()
for i, class_name in enumerate(classes):
    print("{:<15} {:<12.3f} {:<12.3f} {:<12.3f} {:<12}".format(class_name, precision[i], recall[i], f1_score[i], support[i]))
print()
print("{:<15} {:<12} {:<12} {:<12.3f} {:<12}".format("accuracy", "", "", accuracy, int(support.sum())))
print("{:<15} {:<12.3f} {:<12.3f} {:<12.3f} {:<12}".format("macro avg", macro_avg_precision, macro_avg_recall, macro_avg_f1_score, int(support.sum())))
print("{:<15} {:<12.3f} {:<12.3f} {:<12.3f} {:<12}".format("weighted avg", weighted_avg_precision, weighted_avg_recall, weighted_avg_f1_score, int(support.sum())))


Confusion Matrix:
[[4 1 1]
 [6 2 2]
 [3 0 6]]

                precision    recall       f1-score     support     

Cat             0.308        0.667        0.421        6           
Fish            0.667        0.200        0.308        10          
Hen             0.667        0.667        0.667        9           

accuracy                                  0.480        25          
macro avg       0.547        0.511        0.465        25          
weighted avg    0.581        0.480        0.464        25          


Задание 3 (50%). Линейная регрессия 
1. Самостоятельно выберите набор данных, над которым вы хотите работать.
2. Загрузите  данные  и  напишите  код,  чтобы  разбить наборна  обучающийи тестовый наборы данных. Выберите как  минимум 3 переменных,  которые вы можете  использовать  для  прогнозирования  значений некоторой  целевой переменной в наборе.
3. Измерьте точность вашей модели на тренировочном и тестовом наборе данных. Обучите простую (одномерную) линейную  регрессию для  каждой  выбранной переменной. Какая из них дает лучший R2?

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Загрузка датасета
data = pd.read_csv("heart_failure_clinical_records_dataset.csv")

# Выбор переменных для прогнозирования (например, 'age', 'ejection_fraction', 'serum_creatinine')
selected_variables = ['age', 'ejection_fraction', 'serum_creatinine']
target_variable = 'DEATH_EVENT'

# Инициализация словаря для хранения оценок R2 для каждой переменной.
r2_scores = {}

# Разделение данные на обучающие и тестовые наборы.
X_train, X_test, y_train, y_test = train_test_split(data[selected_variables], data[target_variable], test_size=0.2, random_state=42)

# Выполнение линейной регрессии и рассчет R2 для каждой переменной.
for variable in selected_variables:
    # Создание модели линейной регрессии
    model = LinearRegression()
    
    # Подгонка модели к обучающим данным
    model.fit(X_train[[variable]], y_train)
    
    # Прогнозирование на основе тестовых данных
    y_pred = model.predict(X_test[[variable]])
    
    # Рассчет R2 
    r2 = r2_score(y_test, y_pred)
    
    # Сохранение R2 в словаре
    r2_scores[variable] = r2

# Нахождение переменной с лучшим показателем R2.
best_variable = max(r2_scores, key=r2_scores.get)

# Вывод оценки R2 для каждой переменной.
print("Показатель R2:")
for variable, r2 in r2_scores.items():
    print(f"{variable}: {r2}")

# Print the variable with the best R2 score
print(f"Переменная с лучшим показателем R2 '{best_variable}' с R2 = {r2_scores[best_variable]:.4f}")


Показатель R2:
age: -0.06354790051894543
ejection_fraction: 0.038053491355620306
serum_creatinine: -0.042224397434794225
Переменная с лучшим показателем R2 'ejection_fraction' с R2 = 0.0381
